In [1]:
import os
import re
import csv
import sys
import ast 
import json
import time
import urllib
import random
import spotipy
import requests
import datetime
import configparser
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# spotifyAPI存取權限
client_id = '1a23ed1ca359485ab81d524206dba0f6'
client_secret = 'af0e14fdb19a49dfb19ef0c1ea1d42ea'
username = 'qnwv65t11cplaz4dikhl4mjgi'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [30]:
# 取得歌手名稱和類型
# https://github.com/Leats/spotify-historymaker/blob/master/spotifyhistorymaker.py
# Bruno Mars = 0du5cEVh5yTK9QJze8zA0C / Adele = 4dpARuHxo51G3z768sgnrY
def get_artist_name_genres(access_token: str, artist_id: str):    
    bearer_token = f'Bearer {access_token}'
    track_headers = {'Accept': 'application/json','Content-Type': 'application/json','Authorization': bearer_token}
    response = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}', headers=track_headers)
    response.raise_for_status()
    getname = response.json()['name']
    getid = response.json()['id']
    getgenres = response.json()['genres']
    
    list_artist_name.append(getname)
    list_artist_id.append(getid)
    list_artist_genres.append(getgenres)
    return list_artist_name,list_artist_id,list_artist_genres

In [28]:
def info2dataframe(list_artist_name,list_artist_id,list_artist_genres):
    df_artist_info = pd.DataFrame(list(zip(list_artist_name,list_artist_id,list_artist_genres)), columns =['Name','Id','Genres'])
    df_artist_info.to_excel('csv/20200303/df_artist_info.xlsx')
    return df_artist_info

In [31]:
# 取得spotify的access token：https://spotify-token-finder.herokuapp.com/
artist_id = ['0du5cEVh5yTK9QJze8zA0C','4dpARuHxo51G3z768sgnrY','6prmLEyn4LfHlD9NnXWlf7']
access_token = 'BQB3GhWzXA5xxmrLNFLJByB4PPZJLRUr2xMAmVAmHI3z5ZTMrlnsoQ_ESklWBnsMqBN-C5JSAUPopd5KI34'
list_artist_name = []
list_artist_id = []
list_artist_genres = []

for i in range(len(artist_id)):
    list_artist_name,list_artist_id,list_artist_genres = get_artist_name_genres(access_token,artist_id[i])
    df_artist_info = info2dataframe(list_artist_name,list_artist_id,list_artist_genres)

In [135]:
# 歌手類型清單
# 1. https://www.reddit.com/r/spotify/comments/6h7u9g/list_of_genre_uris/
# 2. http://everynoise.com/everynoise1d.cgi?root=pop&scope=all

# 把編號1網址的所有類型爬取回來，建立類型的dataframe
def spotify_genres(url):
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    tables = soup.find('table')
    return tables

# 所有的歌手類型建立txt檔
def spotify_genres_ouput(tables):
    data=[]
    genres_row = tables.findAll('tr')
    with open('csv/20200303/artist_genres.csv','w',newline='') as f:
        writeCsv = csv.writer(f)
        writeCsv.writerow(['Genres'])
        for i in range(0,len(genres_row)):
            td = genres_row[i].findAll('td')
            genres = td[2].getText()
            genres = genres.replace(';','')
            f.write(genres.encode("utf8").decode("cp950", "ignore"))
            data.append(genres.encode("utf8").decode("cp950", "ignore"))
            if i < len(genres_row)-1:
                f.write('\n')
    f.close()
    print('Genres Count:',len(data))
    return data

In [136]:
url = 'http://everynoise.com/everynoise1d.cgi?root=pop&scope=all'
tables = spotify_genres(url)
data = spotify_genres_ouput(tables)

Genres Count: 4009


In [123]:
df_artist_genres = pd.read_csv('csv/20200303/artist_genres.csv')
df_artist_genres

,Genres
0,pop
1,dance pop
2,post-teen pop
3,electropop
4,pop rap
...,...
4004,polish classical
4005,italian choir
4006,classical guitar quartet
4007,wind quintet


In [141]:
# 匯入欲刪除類型的字串
f = open('csv/20200303/delete_string.txt','r',encoding="utf-8") 
lines = f.readlines()
string_list = []
for line in lines:
    string_list.append(line.replace('\n',''))
    #print(line.replace('\n',''))
string = ('|'.join(string_list)).lower()
string

"ambient|dub|electronic rock|sampledelia|chiptune|industrial|hi-nrg|minimal wave|synth-pop|electropop|electropunk|breakbeat|dubtronica|electro|freestyle|electro-industrial|eurodance|hardcoreitalo disco|house|techno|digital hardcore|downtempo|drum and bass|dubstep|electroclash|electro hop|ethnic electronica|folktronica|funktronica|glitch|goa|hardstyle|intelligent dance music|jungle music|livetronica|mákina|nu skool breaks|progressive electronica|trance|uk garage|bass music|bassline|crunk|chillwave|electro house|grime|hauntology|synthwave|uk funky|witch house|big room house|future bass|hardvapour|trap edm|vaporwave|chill-out music|contemporary r&b|disco|post-disco|garage house|electronics in rock music|list of electronic music genres|list of hip hop genres|plunderphonics|turntablism hip-hop music|video game music|alternative hip hop|bouncechicano rap|chopped and screwed|chopper|christian hip hop|cloud rap|comedy hip hop|crunk|dirty rap|drill|east coast|electro|freestyle|freestyle rap|gan

In [142]:
# 篩選去除有rock、r&b、rap、chinese
# 篩選dataframe中含有特定字元的欄位，'|'進行多個條件篩選
df_artist_genres = pd.read_csv('csv/20200303/artist_genres.csv')
df_artist_genres_select = df_artist_genres.loc[~df_artist_genres['Genres'].str.contains(string)].reset_index(drop=True)
df_artist_genres_select.to_excel('csv/20200303/artist_genres_select.xlsx', encoding='utf_8_sig')
df_artist_genres_select

,Genres
0,pop
1,dance pop
2,post-teen pop
3,urban contemporary
4,indie poptimism
...,...
1725,swedish choir
1726,cathedral choir
1727,atmosphere
1728,wind quintet
